## Яндекс Практикум, курс "Инженер Машинного Обучения" (2024 г.)
## Проект 4-го спринта: "Создание рекомендательной системы"
## Ноутбук для генерации оффлайн-рекомендаций
***

### Этап 3. Оффлайн-рекомендации

__Постановка задачи__

Подготовьте данные для обучения:
- Загрузите ранее сохранённые данные.
- Разбейте данные на тренировочную и тестовую выборки: в тренировочную отнесите все данные до 16 декабря 2022 года (не включительно).

Рассчитайте рекомендации нескольких типов, сохраните каждый тип в своём файле:
- топ популярных — в `top_popular.parquet`,
- персональные (при помощи ALS) — в `personal_als.parquet`,
- похожие треки или i2i (при помощи ALS) — в `similar.parquet`.

Постройте ранжирующую модель, которая использует минимум три признака. Отранжируйте рекомендации, а итоговые сохраните в `recommendations.parquet`.

Проверьте оценку качества трёх типов рекомендаций: 
1. топ популярных;
2. персональных, полученных при помощи ALS;
3. итоговых (по метрикам: recall, precision, coverage, novelty).

Приведите значения метрик в README.

In [1]:
import logging
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import joblib
import ipywidgets

from sklearn.metrics import accuracy_score

In [2]:
import sys

# Проверяем, в каком окружении работаем
print(sys.executable)

/home/mle-user/.venv/bin/python


In [3]:
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning, FutureWarning))

In [7]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

Загружаем основные данные

In [4]:
items = pd.read_parquet("items.parquet")
events = pd.read_parquet("events.parquet")

Разбиваем данные на тренировочную и тестовую выборки (в тренировочную относим все данные до 16 декабря 2022 года не включительно).

In [6]:
# Зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2022-12-16")

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx] 
events_test = events[~train_test_global_time_split_idx]

__Топ популярных рекомендаций__

Загружаем ранее сохраненные топ-треки и оцениваем их качество рекомендаций.

In [7]:
top_k_pop_items = pd.read_parquet("top_popular.parquet")

In [8]:
# Идентификаторы уникальных пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()

In [9]:
# Идентификаторы "холодных" пользователей
cold_users = users_test[~users_test.isin(users_train)]

In [21]:
# Формируем колонку prediction
top_k_pop_items['prediction'] = 1

# Объединяем события "холодных" пользователей с топ-треками
cold_users_events = \
    events_test[events_test["user_id"].isin(cold_users)] \
        .merge(top_k_pop_items, on="item_id", how="left")

# Если топ-треки не попали в тестовую выборку для холодных пользователей, то prediction  = 0
cold_users_events['prediction'].fillna(0, inplace=True)

# Формируем колонку target
cold_users_events['target'] = 1

In [23]:
# Считаем точность рекомендаций
accuracy = accuracy_score(cold_users_events['prediction'], cold_users_events['target'])
print(accuracy)

0.05650518569721842


Посчитаем покрытие холодных пользователей рекомендациями

In [24]:
cold_users_hit_ratio = cold_users_events.groupby("user_id").agg(hits=("prediction", 'sum'))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

Доля пользователей без релевантных рекомендаций: 0.42
Среднее покрытие пользователей: 3.53


__Персональные рекомендации (при помощи ALS)__

Для справки: [Техническая документация на библиотеку implicit](https://benfred.github.io/implicit/api/models/cpu/als.html)


Перекодируем идентификаторы `user_id` и `item_id`

In [10]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train = events_train.merge(items[["item_id", "item_id_enc"]], on="item_id", how='left') 
events_test = events_test.merge(items[["item_id", "item_id_enc"]], on="item_id", how='left') 

Создаем sparse-матрицу в формате CSR ([документация](https://docs.scipy.org/doc/scipy/reference/sparse.html))

In [ ]:
# Формируем колонку target
events_train['target'] = 1

In [13]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["target"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

Имея подготовленную матрицу взаимодействий, перейдём к третьему шагу — создадим ALS-модель. Для примера возьмём количество латентных факторов для матриц $P и Q$ равным 50. 

In [11]:
# Код для создания и тренировки модели
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

c:\Users\Kirill_Nosov\_Repos\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Kirill_Nosov\_Repos\.venv\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 50/50 [03:35<00:00,  4.32s/it]


In [12]:
# Сохраняем als-модель в файл

with open('models/als_model.pkl', 'wb') as fd:
    joblib.dump(als_model, fd)

Чтобы получить рекомендации для пользователя с помощью модели ALS, используем функцию ниже

In [19]:
def get_recommendations_als(user_item_matrix, als_model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = als_model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations

__Факультативное задание__

Используя get_recommendations_als, напишите код, который позволит для случайного пользователя просмотреть рекомендации в удобном формате: 
- история с именами авторов и названием книг,
- рекомендации с именами авторов и названием книг, seen-признаком (взаимодействовал ли уже пользователь с рекомендованной книгой).

Проанализируйте, релевантны ли рекомендации имеющейся истории.

In [1]:
# TODO

Получим рекомендации для всех имеющихся пользователей. Выполните код ниже.

In [20]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, 
    N=100)

Код возвращает рекомендации как список списков, это не очень удобно. Преобразуем его в более удобный формат — табличный.

In [21]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

In [37]:
# Сохраним полученные рекомендации в файл
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("personal_als.parquet")

Оценим метрику ранжирования [NDCG](https://machinelearninginterview.com/topics/machine-learning/ndcg-evaluation-metric-for-recommender-systems)
на тестовой выборке. Она принимает значение от 0 (предлагаемый порядок никак не соответствует истинному) до 1 (предлагаемый порядок в точности соответствует истинному). 

Для удобства оценки добавим в датафрейм с рекомендациями истинные оценки из тестовой выборки:

In [22]:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "rating"]]
               .rename(columns={"rating": "rating_test"}), 
           on=["user_id", "item_id"], how="left")
)

Подсчитать метрику NDCG для одного пользователя поможет готовая реализация из scikit-learn:

In [23]:
import sklearn.metrics

def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg

Умея считать NDCG для одного пользователя, посчитаем данную метрику, например, для k=5 для всех пользователей из тестовой выборки. В результате каждому пользователю будет соответствовать одно значение NDCG@5. Запись “NDCG@5” означает, что метрика NDCG считается для пяти айтемов.

In [24]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))

Имея ряд значений NDCG@5 по пользователям, можно посчитать её среднее значение по всем пользователям. Должно получиться  0.9759

In [25]:
ndcg_at_5_scores.mean()

0.9759562997103215

__Факультативное задание__

Оцените, для какой доли пользователей удалось посчитать метрику NDCG.

In [2]:
# TODO

__Рекомендации I2I__

Рекомендации вида I2I — это рекомендации похожих объектов. Например, к некоторой книге порекомендовать список похожих книг. Рассматриваемая реализация ALS позволяет получить такие рекомендации при помощи метода `similar_items` ([описание](https://benfred.github.io/implicit/api/models/cpu/als.html)).

__Факультативное задание__

Используя метод  similar_items, получите и оцените рекомендации для нескольких айтемов. Проанализируйте адекватность результатов.

In [ ]:
# TODO

### Тема 3, Урок 3: Контентные рекомендации

Построим контентные рекомендации для нашего книжного сервиса. Это можно сделать на основе свойства genre_and_votes — доля голосов в пользу отнесения книги к конкретному жанру. 

Перед генерацией рекомендаций нужно подготовить данные.

__Составим список всех жанров__

Составим список всех возможных жанров по всем айтемам. Потом по нему будем составлять вектор с весами по жанрам для каждой книги. Из любопытства составим список жанров с подсчётом количества голосов по каждому из них.
Сначала преобразуем значения genre_and_votes из текстового представления в тип в Python:

In [26]:
items["genre_and_votes"] = items["genre_and_votes"].apply(eval)

__Задание 1 из 4__

Теперь составьте список жанров с долями голосов по ним в genres. Дополните и выполните следующий код:

In [27]:
def get_genres(items):

    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = {}
    
    for k, v, in items.iterrows():
        genre_and_votes = v["genre_and_votes"] # ваш код здесь
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += votes # ваш код здесь
            except KeyError:
                genres_counter[genre] = 0

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"
    
    return genres

genres = get_genres(items)

Результат выполнения кода — список жанров с долями голосов по ним в genres. Посмотрим на самые популярные жанры:

Проверка: На 5-м по популярности месте должен находиться жанр - `Romance`,
на 1-м месте с отрывом по количеству голосов почти в три раза должен быть жанр `Fantasy`.

In [28]:
genres["score"] = genres["votes"] / genres["votes"].sum()
genres.sort_values(by="score", ascending=False).head(10)

,name,votes,score
genre_id,,,
25,Fantasy,6850060,0.149651
1,Fiction,6406256,0.139955
38,Classics,3414934,0.074605
18,Young Adult,3296951,0.072027
34,Romance,2422614,0.052926
5,Nonfiction,1737406,0.037957
16,Historical-Historical Fiction,1531205,0.033452
20,Mystery,1371196,0.029956
24,Science Fiction,1218917,0.026629


__Подготовим матрицы__

При работе с контентными свойствами нередко используют матрицы больших размерностей.
При этом они будут разреженными. Чтобы сэкономить память, используем sparse-формат.

В текущей задаче нам необходимо составить  матрицу, в которой строки соответствуют книгам, а столбцы — жанрам (в том порядке, в котором они указаны в genres), на пересечении — доля голосов.

__Задание 2 из 4__

Функция в коде ниже строит матрицу вида «книга-жанр». Изучите её. Подумайте, что будет соответствовать столбцам матрицы. 

In [29]:
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr

Получим матрицу с весами по жанрам для каждой книги:

In [30]:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

Аналогичным образом получим матрицу с весами по жанрам для какого-нибудь пользователя, например, для пользователя с идентификатором 1000010.

__Задание 3 из 4__

Дополните и выполните код ниже, чтобы получить описанную матрицу. 

Проверка: для выбранного пользователя в user_items_genres_csr должно получиться 149 существующих элементов.

In [47]:
user_id = 1000010
user_events = events_train.query("user_id == @user_id")[["item_id", "rating"]]
user_items = items[items["item_id"].isin(user_events["item_id"])]

user_items_genres_csr = get_item2genre_matrix(genres, user_items) # ваш код здесь
user_items_genres_csr

<22x815 sparse matrix of type '<class 'numpy.float64'>'
	with 149 stored elements in Compressed Sparse Row format>

На практике часто пользователь явно указывает предпочтения в своём профиле. У нас таких данных нет, поэтому предпочтения пользователя по жанрам вычислим автоматически на основе его истории поведения.

In [48]:
# вычислим склонность пользователя к жанрам как среднее взвешенное значение популяции на его оценки книг.

# преобразуем пользовательские оценки из списка в вектор-столбец
user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)

user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0))

Посмотрим, какие жанры больше всего нравятся пользователю:

In [49]:
# выведем список жанров, которые предпочитает пользователь

user_genres = genres.copy()
user_genres["score"] = np.ravel(user_genres_scores)
user_genres = user_genres[user_genres["score"] > 0].sort_values(by=["score"], ascending=False)

user_genres.head(5)

,name,votes,score
genre_id,,,
1,Fiction,6406256,0.195253
38,Classics,3414934,0.096687
25,Fantasy,6850060,0.074261
24,Science Fiction,1218917,0.045902
5,Nonfiction,1737406,0.044359


__Построим рекомендации__

Теперь рассчитаем рекомендации на основе двух объектов:
* all_items_genres_csr — матрица распределения интересов всех пользователей по жанрам. Строка — вектор распределения интересов всех пользователей по жанрам для одного объекта.
* user_genres_scores — вектор интересов пользователя по жанрам.

Рекомендации будем рассчитывать с помощью косинусного сходства между user_genres_scores и всеми
строками матрицы all_items_genres_csr.

__Задание 4 из 4__

Получите наиболее релевантные рекомендации для пользователя. Дополните код так, чтобы переменная top_k_indices заполнялась индексами соответствующих книг. Для этого удобно использовать np.argsort от similarity_scores, подсчитанной для всех книг.

In [50]:
from sklearn.metrics.pairwise import cosine_similarity

# вычисляем сходство между вектором пользователя и векторами по книгам
similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

# преобразуем в одномерный массив
similarity_scores = similarity_scores.flatten()

# получаем индексы top-k (по убыванию значений), по сути, индексы книг (encoded)
k = 5
top_k_indices = np.argsort(-similarity_scores)[:k] # ваш код здесь 

После вычисления top_k_indices по полученным индексам извлеките список объектов, которые могут быть интересны пользователю, при помощи кода:

In [51]:
selected_items = items[items["item_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
   display(selected_items[["author", "title", "genre_and_votes"]])

,author,title,genre_and_votes
564712,Ray Bradbury,"Farewell Summer (Green Town, #3)","{'Fiction': 170, 'Fantasy': 72, 'Science Fiction': 72, 'Classics': 52}"
1358935,John Fowles,The Magus,"{'Fiction': 1204, 'Classics': 421, 'Fantasy': 228, 'Mystery': 203, 'Literature': 167}"
80465,G.K. Chesterton,The Napoleon of Notting Hill,"{'Fiction': 166, 'Classics': 88, 'Fantasy': 44, 'Humor': 22, 'Literature': 20}"
1168335,Ray Bradbury,"Dandelion Wine (Green Town, #1)","{'Fiction': 1438, 'Classics': 914, 'Science Fiction': 529, 'Fantasy': 456, 'Young Adult': 212}"
2244467,Samuel Butler,"Erewhon (Erewhon , #1)","{'Fiction': 162, 'Classics': 139, 'Science Fiction': 60, 'Fantasy': 55}"


Самым преобладающим жанром в полученных рекомендациях явл-ся Fiction. За него отдано наибольшее количество голосов. Это согласуется и со списком жанров, предпочитаемых пользователем.

__Факультативное задание__
1. Получите по алгоритму выше рекомендации для нескольких пользователей, просмотрите их на экране. Подумайте, насколько релевантны и интересны полученные рекомендации пользователям.
2. Попробуйте использовать другую меру сходства для получения рекомендаций, например, евклидово расстояние. Проанализируйте, отличаются ли рекомендации от предыдущих. Подумайте почему.
3. Задайте собственные предпочтения для наиболее популярных жанров. Посмотрите рекомендации для себя. Прочитали ли бы вы рекомендованные книги?

In [32]:
from sklearn.metrics.pairwise import cosine_similarity


def get_recommendations_by_genres(user_id, items, events, genres, all_items_genres_csr, n=5):

    """ возвращает n рекомендаций для user_id на основе предпочитаемых им жанров"""
    
    user_events = events.query("user_id == @user_id")[["item_id", "rating"]]
    user_items = items[items["item_id"].isin(user_events["item_id"])]

    #user_items_genres_csr = get_item2genre_matrix(genres, user_items) # Авторский вариант
    user_items_genres_csr = all_items_genres_csr[user_items['item_id_enc']] # Мой вариант, рез-т тот же
    
    # вычислим склонность пользователя к жанрам как среднее взвешенное значение популяции на его оценки книг.

    # преобразуем пользовательские оценки из списка в вектор-столбец
    user_ratings = user_events["rating"].to_numpy() / 5
    user_ratings = np.expand_dims(user_ratings, axis=1)

    user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)
    user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0))

    # вычисляем сходство между вектором пользователя и векторами по книгам
    similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

    # преобразуем в одномерный массив
    similarity_scores = similarity_scores.flatten()

    # получаем индексы top-n (по убыванию значений), по сути, индексы книг (encoded)
    top_n_indices = np.argsort(-similarity_scores)[:n] 

    selected_items = items[items["item_id_enc"].isin(top_n_indices)]
    
    return selected_items['item_id'].to_list(), similarity_scores[top_n_indices].tolist()

In [33]:
# Для примера взял только первые 1000 юзеров

users_ids = events_train['user_id'].unique()[:1000] 

n = 5
items_ids = []
scores = []
for i, user_id in enumerate(users_ids):
    user_items_ids, user_scores = get_recommendations_by_genres(user_id, items, events_train, genres, all_items_genres_csr, n)
    items_ids.append(user_items_ids)
    scores.append(user_scores)

    if i >= 1000:
        break

content_recommendations = pd.DataFrame({
    "user_id": users_ids,
    "item_id": items_ids, 
    "score": scores})

content_recommendations = content_recommendations.explode(["item_id", "score"], ignore_index=True)

In [34]:
content_recommendations

,user_id,book_id,score
0,1000000,11841,0.931126
1,1000000,18130,0.920573
2,1000000,24761,0.919324
3,1000000,84981,0.9118
4,1000000,7052617,0.906302
...,...,...,...
4995,1001004,10410278,0.959062
4996,1001004,12256544,0.956481
4997,1001004,14743247,0.955078
4998,1001004,16070143,0.95395


In [35]:
# Сохраним полученные рекомендации в файл
#content_recommendations.to_parquet("content_recommendations.parquet")

### Тема 3, Урок 4: Валидация

Посчитаем recall и precision для ALS-рекомендаций (als_recommendations). Для этого события в тестовой выборке и рекомендации для одних и тех же пользователей разметим признаками:
* gt (ground truth): объект есть в тестовой выборке;
* pr (predicted): объект есть в рекомендациях.

Теперь разметим признаки бинарной классификации:
* TP: объект есть и в тестовой выборке, и в рекомендациях (истинная рекомендация),
* FP: объекта нет в тестовой выборке, но он есть в рекомендациях (ложноположительная рекомендация),
* FN: объект есть в тестовой выборке, но его нет в рекомендациях (ложноотрицательная рекомендация)

In [31]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["item_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")    

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()
    
    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common

Обработаем ALS-рекомендации для подсчёта метрик для 5 лучших рекомендаций:

In [32]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(events_train,
                                                                        events_test, 
                                                                        als_recommendations,
                                                                        top_k=5)

Common users: 123223


__Задание__

Дополните код функции compute_cls_metrics для расчёта recall. Получите значения метрик precision@5, recall@5

In [33]:
def compute_cls_metrics(events_recs_for_binary_metrics):
    
    groupper = events_recs_for_binary_metrics.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum() / (groupper["tp"].sum() + groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum() / (groupper["tp"].sum() + groupper["fn"].sum()) # ваш код здесь
    recall = recall.fillna(0).mean()

    return precision, recall

Recall@5 должен быть равен 0.014

In [34]:
precision, recall = compute_cls_metrics(events_recs_for_binary_metrics)
print(round(precision, 3), round(recall, 3))

0.008 0.014


__Факультативное задание__

Посчитайте метрики precision@10, recall@10. Сравните их значения со значениями для precision@5, recall@10. Подумайте о причинах таких отличий.

In [107]:
events_recs_for_binary_metrics_10 = process_events_recs_for_binary_metrics(events_train,
                                                                           events_test, 
                                                                           als_recommendations,
                                                                           top_k=10)

Common users: 123223


In [109]:
precision_10, recall_10 = compute_cls_metrics(events_recs_for_binary_metrics_10)
print(round(precision_10, 3), round(recall_10, 3))

0.009 0.031


## Тема 4: Двухстадийный подход

### Тема 4, Урок 2: Специфичные метрики

__Задание 1 из 2__

Для рекомендаций, сохранённых в переменной als_recommendations, посчитайте покрытие по объектам согласно формуле выше. При этом используйте весь топ-100 рекомендаций. Ответ округлите до сотых.

In [12]:
# Загружаем als-рекомендации
als_recommendations = pd.read_parquet("als_recommendations.parquet")
als_recommendations.head()

,user_id,book_id,score
0,1000000,3,0.990942
1,1000000,15881,0.896617
2,1000000,5,0.864405
3,1000000,6,0.822254
4,1000000,2,0.774096


Должно получиться 0.09

In [35]:
# расчёт покрытия по объектам
cov_items = als_recommendations['item_id'].nunique() / len(items) # ваш код здесь
print(f"{cov_items:.2f}")

0.09


__Задание 2 из 2__

Посчитайте среднее Novelty@5 для als_recommendations, ответ округлите до сотых. 

Для этого: 
* разметьте каждую рекомендацию в als_recommendations булевым признаком read (False — пользователь не читал книгу, True — пользователь читал книгу), используя events_train,
* посчитайте Novelty@5 для каждого пользователя,
* посчитайте среднеарифметическое для полученных значений Novelty@5.

Завершите выполнение такого расчёта с помощью кода ниже, должно получиться 0.61

In [36]:
# разметим каждую рекомендацию признаком read
events_train["read"] = True
als_recommendations = als_recommendations.merge(events_train[['read', "user_id", "item_id"]], on=["user_id", "item_id"], how="left") # ваш код здесь
als_recommendations["read"] = als_recommendations["read"].fillna(False).astype("bool")

# проставим ранги
als_recommendations = als_recommendations.sort_values(by='score', ascending=False) # ваш код здесь
als_recommendations["rank"] = als_recommendations.groupby("user_id").cumcount() + 1

# посчитаем novelty по пользователям
novelty_5 = (1 - als_recommendations.query("rank <= 5").groupby("user_id")["read"].mean())

# посчитаем средний novelty
# ваш код здесь
novelty_5_mean = novelty_5.mean()
print(f"{novelty_5_mean:.2f}")

0.61


### Тема 4, Урок 3: Двухстадийный подход

__Задание 1 из 6__

Используем отложенную тестовую часть данных — назовём её events_test — для получения двух новых частей данных:
* одна, составляющая первые 45 дней, будет использоваться для таргетов,
* другая, состоящая из 45 последних дней, будет новой тестовой выборкой.


Завершите код так, чтобы в events_labels оказалась первая часть данных, а в events_test_2 — вторая.

In [37]:
# задаём точку разбиения
split_date_for_labels = pd.to_datetime("2017-09-15").date()

split_date_for_labels_idx = events_test["started_at"] < split_date_for_labels
events_labels = events_test[split_date_for_labels_idx].copy() # ваш код здесь
events_test_2 = events_test[~split_date_for_labels_idx].copy() # ваш код здесь

Проверим кол-во уникальных пользователей в events_labels, должно получиться 99849

In [38]:
events_labels['user_id'].nunique()

99849

__Подготовка кандидатов для обучения__

Подготовим список кандидатов для обучения ранжирующей модели. В качестве кандидатогенераторов возьмём ALS и контентную модель на основе жанровых предпочтений, известных нам из прошлых уроков. Рекомендации от них были заранее подготовлены и сохранены в файлах als_recommendations.parquet и content_recommendations.parquet в директории candidates/training. Подготовка заключается в объединении списков рекомендаций по совпадению user_id, item_id. 

__Задание 2 из 6__

Объедините имеющихся кандидатов по совпадению user_id, item_id в один список.

In [25]:
os.makedirs('./candidates/training/', exist_ok=True)
os.getcwd()

'c:\\Users\\Kirill_Nosov\\_Repos\\mle_projects\\mle-recsys'

In [24]:
#!python -m wget -o candidates/training/content_recommendations.parquet https://storage.yandexcloud.net/mle-data/candidates/training/content_recommendations.parquet
#!python -m wget -o candidates/training/als_recommendations.parquet https://storage.yandexcloud.net/mle-data/candidates/training/als_recommendations.parquet


Saved under candidates/training/als_recommendations.parquet


In [39]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations = pd.read_parquet("candidates/training/als_recommendations.parquet")
content_recommendations = pd.read_parquet("candidates/training/content_recommendations.parquet")

In [40]:
candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer") # ваш код здесь

Проверим кол-во записей в candidates, должно получиться 82993094

In [41]:
len(candidates)

82993094

__Таргеты__

Кандидаты готовы, теперь можно сформировать для них таргеты. Каждая строка, которая формируется для обучения модели ранжирования, — это строка, ассоциированная с парой «пользователь — объект». Положительным таргетом, то есть 1, будем считать любое значимое действие пользователя с объектом, а отрицательным, то есть 0, — его отсутствие. В таком случае модель ранжирования обучается предсказывать вероятность взаимодействия пользователя с объектом, рекомендуемым кандидатогенератором.

При этом следует контролировать баланс классов и при необходимости проводить негативное семплирование: уменьшать количество строк с отрицательным таргетом, чтобы не было сильного дисбаланса классов.
Для негативного семплирования возможны различные стратегии: отбрасывание случайных записей равномерно, пропорционально их популярности, с различными весами и так далее.

Добавим в candidates таргеты, используя историю взаимодействий в events_labels. 

__Задание 3 из 6__

Дополните код ниже. В candidates добавьте колонку target со значениями:
- 1 для тех item_id, которые пользователь прочитал (положительный пример).
- 0 — для всех остальных (негативный пример).

В candidates_for_train отберите все положительные примеры, а также не менее четырёх негативных примеров для каждого пользователя в положительных примерах.

In [42]:
# добавляем таргет к кандидатам со значением:
# — 1 для тех item_id, которые пользователь прочитал
# — 0, для всех остальных 

events_labels["target"] = 1
candidates = candidates.merge(events_labels[["user_id", "item_id", "target"]], 
                              on=["user_id", "item_id"], # ваш код здесь 
                              how='left') 
candidates["target"] = candidates["target"].fillna(0).astype("int")

# в кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

# для каждого пользователя оставляем только 4 негативных примера
negatives_per_user = 4
candidates_for_train = pd.concat([
    candidates_to_sample.query("target == 1"), # ваш код здесь
    candidates_to_sample.query("target == 0") \
        .groupby("user_id") \
        .apply(lambda x: x.sample(negatives_per_user, random_state=0))
    ])

Проверим кол-во записей в candidates_for_train, должно получтиься 213708

In [43]:
len(candidates_for_train)

213708

__Обучение модели__

In [29]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6526062	total: 175ms	remaining: 2m 54s
100:	learn: 0.5119048	total: 2.6s	remaining: 23.1s
200:	learn: 0.5112058	total: 5.07s	remaining: 20.2s
300:	learn: 0.5105778	total: 7.59s	remaining: 17.6s
400:	learn: 0.5100299	total: 10.1s	remaining: 15.1s
500:	learn: 0.5095632	total: 12.5s	remaining: 12.4s
600:	learn: 0.5091520	total: 14.9s	remaining: 9.88s
700:	learn: 0.5087528	total: 17.3s	remaining: 7.36s
800:	learn: 0.5084194	total: 19.9s	remaining: 4.94s
900:	learn: 0.5080962	total: 22.6s	remaining: 2.48s
999:	learn: 0.5078292	total: 25.2s	remaining: 0us


In [30]:
os.makedirs('models/', exist_ok=True)
cb_model.save_model('models/cb_model.cbm')

Теперь можно получить финальные рекомендации, используя обученную модель, сохранённую в переменной cb_model.

__Подготовка кандидатов для рекомендаций__

Представим, что натренированная модель используется только некоторое время спустя, когда уже появились новые рекомендации (кандидаты) от базовых генераторов, обученных на объединении событий из events_train и events_label. Иными словами, когда события из events_label уже стали частью тренировочного набора данных. Эти новые рекомендации были заранее подготовлены и сохранены в файлах als_recommendations.parquet и content_recommendations.parquet в директории candidates/inference. Используем их для составления нового списка кандидатов candidates_to_rank, который понадобится готовой ранжирующей модели. 

In [82]:
os.makedirs('candidates/inference/', exist_ok=True)

In [84]:
!python -m wget -o candidates/inference/content_recommendations.parquet https://storage.yandexcloud.net/mle-data/candidates/inference/content_recommendations.parquet
!python -m wget -o candidates/inference/als_recommendations.parquet https://storage.yandexcloud.net/mle-data/candidates/inference/als_recommendations.parquet


Saved under candidates/inference/content_recommendations.parquet

Saved under candidates/inference/als_recommendations.parquet


In [44]:
# загружаем новые рекомендации от двух базовых генераторов
als_recommendations_2 = pd.read_parquet("candidates/inference/als_recommendations.parquet")
content_recommendations_2 = pd.read_parquet("candidates/inference/content_recommendations.parquet")

__Задание 4 из 6__

Дополните код ниже так, чтобы в candidates_to_rank попали кандидаты от обоих базовых генераторов подобно тому, как это было сделано для фазы тренировки выше.

In [45]:
candidates_to_rank = pd.merge(
    als_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer"
) # ваш код здесь

# оставляем только тех пользователей, что есть в тестовой выборке, для экономии ресурсов
candidates_to_rank = candidates_to_rank[candidates_to_rank["user_id"].isin(events_test_2["user_id"].drop_duplicates())]

Проверим кол-во записей в candidates_to_rank, должно получиться 14517152

In [46]:
len(candidates_to_rank)

14517152

__Ранжирование кандидатов для рекомендаций__

Применим обученную ранжирующую модель к кандидатам для рекомендаций. Таргет уже не нужен, поскольку мы будем применять модель в режиме инференса.

__Задание 5 из 6__

Дополните код для того, чтобы вызвать модель и получить для каждого пользователя топ-100 рекомендаций — значение rank нужно выставить не более ста.

In [47]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score']

cb_model = CatBoostClassifier()
cb_model.load_model('models/cb_model.cbm')

In [48]:
inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

# для каждого пользователя проставляем rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = candidates_to_rank.groupby('user_id').cumcount() + 1 # ваш код здесь

# Мой вариант (в этом случае предварительная сортировка не нужна)
# candidates_to_rank["rank"] = candidates_to_rank.groupby('user_id')['cb_score'].rank('first', ascending=False).astype('int') # ваш код здесь

max_recommendations_per_user = 100
final_recommendations = candidates_to_rank.query('rank <= @max_recommendations_per_user') # ваш код здесь

Проверим кол-во записей в final_recommendations, должно получиться 7519400

In [49]:
len(final_recommendations)

7519400

__Валидация__

Для валидации применимы всё те же метрики из предыдущих уроков. Считать их можно, как мы уже упоминали, как для базовых генераторов, так и для ранжирующей модели.

Подобным же образом при валидации можно визуально просматривать как рекомендации от базовых моделей, так и финальные. Такой ручной просмотр по пользователям может подсказать, какой смысловой вклад вносят те или иные кандидатогенераторы в итоговые рекомендации.

__Задание 6 из 6__

Посчитайте метрики recall и precision.

1. Используйте полученные рекомендации final_recommendations, отложенную тестовую выборку events_test_2, созданные в уроке «Валидация» предыдущей темы. 
2. А также функции process_events_recs_for_binary_metrics и compute_cls_metrics.

In [50]:
events_inference = pd.concat([events_train, events_labels])

cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2, # ваш код здесь
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5)

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5) # ваш код здесь

Common users: 75194


Проверим значение метрики recall, должно получиться 0.016

In [51]:
print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

precision: 0.006, recall: 0.015


### Тема 4, Урок 4: Как использовать признаки для улучшения рекомендаций

В предыдущем уроке мы рассмотрели, как использовать градиентный бустинг для ранжирования кандидатов от базовых алгоритмов. Признаками, или фичами, были оценки из базовых алгоритмов. На деле же ничто не помешает нам придумать и использовать признаки другой природы, чтобы повысить качество ранжирования — а следовательно, и качество рекомендаций.

### Признаки объектов
__Задание 1 из 6__

Посчитаем новый признак — «возраст» книги на основе года публикации. Назовём его age. 

Используя обновлённый справочник объектов items, добавьте признаки возраста age и средней популярности average_rating к кандидатам для тренировки модели candidates_for_train и к кандидатам для ранжирования candidates_to_rank.

In [52]:
items["age"] = 2018 - items["publication_year"]
invalid_age_idx = items["age"] < 0
items.loc[invalid_age_idx, "age"] = np.nan
items["age"] = items["age"].astype("float")

candidates_for_train = candidates_for_train.merge(items[['item_id', 'age', 'average_rating']], on='item_id', how='left') # ваш код здесь
candidates_to_rank = candidates_to_rank.merge(items[['item_id', 'age', 'average_rating']], on='item_id', how='left') # ваш код здесь

Проверим медианный возраст книги для candidates_to_rank, должно получиться 10

In [53]:
candidates_to_rank['age'].median()

7.0

### Признаки пользователей
__Задание 2 из 6__

Используя события в events_train и events_inference, посчитайте и добавьте признаки пользователей к кандидатам в candidates_for_train и candidates_to_rank соответственно:
* reading_years — длительность истории пользователя,
* books_read — количество книг, прочитанных за всё время,
* books_per_year — среднее количество прочитанных книг в год,
* rating_avg — средняя оценка,
* rating_std — дисперсия оценок.

In [54]:
def get_user_features(events):
    """ считает пользовательские признаки """
    
    user_features = events.groupby("user_id").agg(
        reading_years=("started_at", lambda x: (x.max() - x.min()).days / 365.25),
        books_read=("is_read", "sum"), # ваш код здесь
        rating_avg=("rating", "mean"),
        rating_std=("rating", "std")
    )
    
    user_features["books_per_year"] = user_features["books_read"] / user_features["reading_years"]
    return user_features
    

user_features_for_train = get_user_features(events_train)
candidates_for_train = candidates_for_train.merge(user_features_for_train, on="user_id", how="left")
  
# оставим только тех пользователей, что есть в тесте, для экономии ресурсов
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test_2["user_id"].drop_duplicates())]

user_features_for_ranking = get_user_features(events_inference) # ваш код здесь
candidates_to_rank = candidates_to_rank.merge(user_features_for_ranking, on="user_id", how="left") # ваш код здесь

Проверим медиану количества прочитанных книг по всем кандидатам в candidates_for_train. Должно получиться 32 (т.е. половина пользователей прочитала не менее 32 книг).

In [55]:
candidates_for_train['books_read'].median()

32.0

### Парные признаки
__Задание 3 из 6__

Используя истории events_train и events_inference, а также ранее полученные артефакты по жанрам книг — словарь жанров genres, оценки книг по жанрам all_items_genres_csr — добавьте парные признаки, по одному на каждый жанр, которые совместно показывают, какие жанры предпочитает пользователь.

Жанровость в данном случае — численный коэффициент принадлежности книги к жанру. Например, если пользователь прочитал три книги, которые с весами 0.3, 0.2, 0.4 из  all_items_genres_csr относятся к Fantasy, то интерес пользователя к Fantasy составляет среднее этих трёх оценок — 0.3.

Для экономии ресурсов возьмём не все жанры, а 10 наиболее популярных. Все остальные отметим как не вошедшие в топ и обозначим как others.

In [56]:
# определяем индексы топ-10 жанров и всех остальных
genres_top_k = 10
genres_top_idx = genres.sort_values("votes", ascending=False).head(genres_top_k).index
genres_others_idx = list(set(genres.index) - set(genres_top_idx))

genres_top_columns = [f"genre_{id}" for id in genres_top_idx]
genres_others_column = "genre_others"
genre_columns = genres_top_columns + [genres_others_column] # ваш код здесь

# составляем таблицу принадлежности книг к жанрам
item_genres = (
    pd.concat([
        # топ жанров
        pd.DataFrame(all_items_genres_csr[:, genres_top_idx].toarray(), columns=genres_top_columns), # ваш код здесь 
        # все остальные жанры
        pd.DataFrame(all_items_genres_csr[:, genres_others_idx].sum(axis=1), columns=[genres_others_column])
        ],
        axis=1)
    .reset_index()
    .rename(columns={"index": "item_id_enc"})
)

# объединяем информацию принадлежности книг к жанрам с основной информацией о книгах
items = items.merge(item_genres, on="item_id_enc", how="left")


def get_user_genres(events, items, item_genre_columns):
    user_genres = (
        events
        .merge(items[["item_id"] + item_genre_columns], on="item_id", how="left")
        .groupby("user_id")[item_genre_columns].mean()
    )
    return user_genres
    

user_genres_for_train = get_user_genres(events_train, items, genre_columns) # ваш код здесь
candidates_for_train = candidates_for_train.merge(user_genres_for_train, on="user_id", how="left")

user_genres_for_ranking = get_user_genres(events_inference, items, genre_columns)
candidates_to_rank = candidates_to_rank.merge(user_genres_for_ranking, on="user_id", how="left")

Проверим медиану жанровости книг в candidates_for_train для жанра “Romance”. При округлении до сотых должно получиться 0.04

In [57]:
# Сначала посмотрим, какой индекс у строчки, соответствующей жанру Romance в таблице genres
genres.sort_values("votes", ascending=False).head(genres_top_k)

,name,votes,score
genre_id,,,
25,Fantasy,6850060,0.149651
1,Fiction,6406256,0.139955
38,Classics,3414934,0.074605
18,Young Adult,3296951,0.072027
34,Romance,2422614,0.052926
5,Nonfiction,1737406,0.037957
16,Historical-Historical Fiction,1531205,0.033452
20,Mystery,1371196,0.029956
24,Science Fiction,1218917,0.026629


In [58]:
round(candidates_for_train['genre_34'].median(), 2)

0.04

### Парные признаки
Вы добавили в candidates_for_train и candidates_to_rank различные признаки. Обучите новую ранжирующую модель, которая их будет учитывать.

__Задание 4 из 6__

Обучите модель, выполнив код ниже

In [84]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score', 
    'age', 'average_rating', 'reading_years', 'books_read', 
    'rating_avg', 'rating_std', 
    'books_per_year'] + genre_columns
target = 'target'

# создаём Pool
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target]
)

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0,
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6484882	total: 42.1ms	remaining: 42.1s
100:	learn: 0.4660990	total: 3.37s	remaining: 30s
200:	learn: 0.4578058	total: 6.58s	remaining: 26.2s
300:	learn: 0.4518839	total: 9.58s	remaining: 22.3s
400:	learn: 0.4469100	total: 14.2s	remaining: 21.2s
500:	learn: 0.4427262	total: 18.8s	remaining: 18.7s
600:	learn: 0.4389780	total: 23.4s	remaining: 15.5s
700:	learn: 0.4354232	total: 27.9s	remaining: 11.9s
800:	learn: 0.4320194	total: 32.5s	remaining: 8.08s
900:	learn: 0.4287993	total: 37.8s	remaining: 4.15s
999:	learn: 0.4258214	total: 42.9s	remaining: 0us


In [85]:
# Сохраняем новую модель с расширенным набором признаков
cb_model.save_model('models/cb_model_2.cbm')

Получим топ-100 самых релевантных рекомендаций для каждого пользователя, используя обученную модель

In [61]:
from catboost import CatBoostClassifier, Pool

cb_model = CatBoostClassifier()
cb_model.load_model('models/cb_model_2.cbm')

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score', 
    'age', 'average_rating', 'reading_years', 'books_read', 
    'rating_avg', 'rating_std', 
    'books_per_year'] + genre_columns

In [62]:
inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1] # ваш код здесь

# для каждого пользователя проставим rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = candidates_to_rank.groupby('user_id').cumcount() + 1 # ваш код здесь

# Мой вариант (в этом случае предварительная сортировка не нужна)
# candidates_to_rank["rank"] = candidates_to_rank.groupby('user_id')['cb_score'].rank('first', ascending=False).astype('int') # ваш код здесь

max_recommendations_per_user = 100
final_recommendations = candidates_to_rank.query("rank <= @max_recommendations_per_user")

Проверим, сколько пользователей попало в final_recommendations. Должно получиться 75194

In [63]:
final_recommendations['user_id'].nunique()

75194

In [1]:
# Сохраним последний вариант персональных рекомендаций в файл
final_recommendations.to_parquet("final_recommendations_feat.parquet")

NameError: name 'final_recommendations' is not defined

Итак, мы получили рекомендации, которые уже должны учитывать не только оценки от базовых генераторов als_score и cnt_score, но и информацию, заложенную в признаках. Посмотрим, помогло ли это повысить качество рекомендаций по метрике recall, по которой мы уже оценивали результаты работы модели в прошлом уроке после внедрения двухстадийного подхода. Напомним, что тогда получилось значение 0.016.

__Задание 5 из 6__

Используя отложенную тестовую выборку events_test_2, посчитайте метрики recall и precision для полученных рекомендаций.

In [64]:
# для экономии ресурсов оставим события только тех пользователей, 
# для которых следует оценить рекомендации
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test_2["user_id"].drop_duplicates())]

cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2, # ваш код здесь
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5 # ваш код здесь
)

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

Common users: 75194


Проверим значение recall. После округления до тысячных должно получиться 0.030

In [65]:
print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

precision: 0.011, recall: 0.029


Т.о., значение recall заметно увеличилось, а это явно говорит о том, что признаки действительно помогли качеству рекомендаций.

### Проверка важности признаков

Любопытно понять, какие признаки вносят наибольший вклад в ранжирование. Алгоритм CatBoost позволяет получить такую информацию (англ. feature importance), которая генерируется во время тренировки модели. Для этого используйте метод get_feature_importance().

__Задание 6 из 6__

Выполните код для получения информации о важности признаков. Выведите список признаков feature_importance в порядке убывания их важности.

In [66]:
feature_importance = pd.DataFrame(cb_model.get_feature_importance(), 
    index=features, 
    columns=["fi"])

feature_importance = feature_importance.sort_values('fi', ascending=False) # ваш код здесь

Проверим, какой у модели самый важный признак. Должен получиться `als_score`

In [67]:
print(feature_importance)

                       fi
als_score       26.540019
age             19.025102
average_rating  15.130070
books_read       7.234967
reading_years    3.333355
cnt_score        3.001030
genre_18         2.898364
genre_1          2.843781
genre_others     2.606153
genre_25         2.359137
genre_34         2.191086
books_per_year   1.814563
rating_avg       1.525892
genre_38         1.519235
genre_33         1.518597
genre_24         1.512781
genre_20         1.454943
genre_16         1.377449
rating_std       1.062938
genre_5          1.050538


## Тема 5: Архитектура рекомендательных систем

### Тема 5, Урок 3: Сервис рекомендаций

__Шаг 1. Шаблон сервиса__

Создадим шаблон сервиса, который пока что умеет только возвращать пустой список.

In [ ]:
# Файл recommendation_service.py (v0)
# Запуск сервиса: uvicorn recommendation_service:app

import logging

from fastapi import FastAPI
from contextlib import asynccontextmanager

logger = logging.getLogger("uvicorn.error")

@asynccontextmanager
async def lifespan(app: FastAPI):
    # код ниже (до yield) выполнится только один раз при запуске сервиса
    logger.info("Starting")
    yield
    # этот код выполнится только один раз при остановке сервиса
    logger.info("Stopping")
    
# создаём приложение FastAPI
app = FastAPI(title="recommendations", lifespan=lifespan)

@app.post("/recommendations")
async def recommendations(user_id: int, k: int = 100):
    """
    Возвращает список рекомендаций длиной k для пользователя user_id
    """

    recs = []

    return {"recs": recs}

__Задание 1 из 3__

Дополните код скрипта и запустите его для получения ответа (в виде пока что пустого списка) от сервиса рекомендаций.

Если сервер uvicorn запущен и нормально работает, то при запуске этого скрипта должен получиться ответ `{'recs': []}`

In [ ]:
# Скрипт для обращения к сервису recommendation_service

import requests

recommendations_url = "http://127.0.0.1:8000/recommendations" # ваш код здесь

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 0}

resp = requests.post(recommendations_url, headers=headers, params=params)

if resp.status_code == 200:
    recs = resp.json()
else:
    recs = []
    print(f"status code: {resp.status_code}")
    
print(recs)

__Шаг 2. Подключение готовых рекомендаций__

Добавим пользы в наш сервис. Для этого сделаем так, чтобы при его запуске загружались уже готовые рекомендации, а затем и отдавались при вызове `/recommendations`. Для удобства основной инструментарий разместим в отдельном классе, который приведён ниже в готовом виде:

In [ ]:
# Класс Recommendations для работы с готовыми рекомендациями

import logging as logger
import pandas as pd

class Recommendations:

    def __init__(self):

        self._recs = {"personal": None, "default": None}
        self._stats = {
            "request_personal_count": 0,
            "request_default_count": 0,
        }

    def load(self, type, path, **kwargs):
        """
        Загружает рекомендации из файла
        """

        logger.info(f"Loading recommendations, type: {type}")
        self._recs[type] = pd.read_parquet(path, **kwargs)
        if type == "personal":
            self._recs[type] = self._recs[type].set_index("user_id")
        logger.info(f"Loaded")

    def get(self, user_id: int, k: int=100):
        """
        Возвращает список рекомендаций для пользователя
        """
        try:
            recs = self._recs["personal"].loc[user_id]
            recs = recs["item_id"].to_list()[:k]
            self._stats["request_personal_count"] += 1
        except KeyError:
            recs = self._recs["default"]
            recs = recs["item_id"].to_list()[:k]
            self._stats["request_default_count"] += 1
        except:
            logger.error("No recommendations found")
            recs = []

        return recs

    def stats(self):
        logger.info("Stats for recommendations")
        for name, value in self._stats.items():
            logger.info(f"{name:<30} {value} ")

Проверьте работу данного класса, используя готовые рекомендации из предыдущих уроков:
- top_recs.parquet — файл с рекомендациями по умолчанию,
- final_recommendations_feat.parquet — файл с персональными рекомендациями.

In [ ]:
!python -m wget -o top_recs.parquet https://storage.yandexcloud.net/mle-data/top_recs.parquet

In [3]:
top_recs = pd.read_parquet("top_recs.parquet")
top_recs

,item_id,users,avg_rating,popularity_weighted,author,title,genre_and_votes,publication_year,score,rank
0,22557272,40690,3.788965,154173.0,Paula Hawkins,The Girl on the Train,"{'Fiction': 9793, 'Mystery': 9190, 'Thriller':...",2015,1.000000,1
1,29056083,25785,3.801784,98029.0,"John Tiffany, Jack Thorne, J.K. Rowling",Harry Potter and the Cursed Child - Parts One ...,"{'Fantasy': 14466, 'Fiction': 4232, 'Young Adu...",2016,0.500000,2
2,18007564,20207,4.321275,87320.0,Andy Weir,The Martian,"{'Science Fiction': 11966, 'Fiction': 8430}",2014,0.333333,3
3,18143977,19462,4.290669,83505.0,Anthony Doerr,All the Light We Cannot See,"{'Historical-Historical Fiction': 13679, 'Fict...",2014,0.250000,4
4,16096824,16770,4.301014,72128.0,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman...",2015,0.200000,5
...,...,...,...,...,...,...,...,...,...,...
95,15704307,5322,4.410936,23475.0,"Brian K. Vaughan, Fiona Staples","Saga, Vol. 1 (Saga, #1)","{'Sequential Art-Graphic Novels': 7803, 'Seque...",2012,0.010417,96
96,22318578,6451,3.626104,23392.0,"Marie Kondō, Cathy Hirano",The Life-Changing Magic of Tidying Up: The Jap...,"{'Nonfiction': 6896, 'Self Help': 2846, 'Audio...",2014,0.010309,97
97,17378508,5284,4.335541,22909.0,Maggie Stiefvater,"Blue Lily, Lily Blue (The Raven Cycle, #3)","{'Fantasy': 5121, 'Young Adult': 3651, 'Fantas...",2014,0.010204,98
98,23848559,5592,4.065629,22735.0,Jenny Lawson,Furiously Happy: A Funny Book About Horrible T...,"{'Nonfiction': 3138, 'Humor': 1914, 'Autobiogr...",2015,0.010101,99


In [6]:
final_recommendations_feat = pd.read_parquet("final_recommendations_feat.parquet")

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Kirill_Nosov\_Repos\.venv\Lib\site-packages\wget.py", line 568, in <module>
    filename = download(args[0], out=options.output)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Kirill_Nosov\_Repos\.venv\Lib\site-packages\wget.py", line 526, in download
    (tmpfile, headers) = ulib.urlretrieve(binurl, tmpfile, callback)
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Kirill_Nosov\anaconda3\Lib\urllib\request.py", line 241, in urlretrieve
    with contextlib.closing(urlopen(url, data)) as fp:
                            ^^^^^^^^^^^^^^^^^^
  File "C:\Users\Kirill_Nosov\anaconda3\Lib\urllib\request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Kirill_Nosov\anaconda3\Lib\urllib\request.py", line

__Задание 2 из 3__

Дополните код и вызовите запущенный сервис рекомендаций, чтобы получить от него пустой (пока) список.

In [ ]:
# Проверяем работу класса Recommendations

rec_store = Recommendations()

rec_store.load(
    "personal",
    'final_recommendations_feat.parquet', # ваш код здесь
    columns=["user_id", "item_id", "rank"],
)
rec_store.load(
    "default",
    'top_recs.parquet', # ваш код здесь
    columns=["item_id", "rank"],
)

rec_store.get(user_id=100, k=5)

Проверим значение у идентификатора объекта (item_id), идущего в списке рекомендаций вторым, для пользователя с идентификатором `1049126` (user_id). Должно получиться `5470`

In [ ]:
rec_store.get(user_id=1049126, k=2)

__Шаг 3. Интеграция__

Интегрируем класс Recommendations в наш сервис для дальнейшего использования. Достаточно перенести код определения класса с импортами (кроме импорта logging) в скрипт с кодом сервиса, а затем разместить код загрузки и получения рекомендаций в соответствующие места:
- код загрузки рекомендаций следует поместить в функцию lifespan,
- код получения рекомендаций — в функцию recommendations,
- чтобы вывести статистику в лог, в функцию lifespan можно поместить вызов rec_store.stats() — он должен вызываться при остановке сервиса.

__Задание 3 из 3__

Интегрируйте код класса `Recommendations` в код сервиса так, чтобы через метод `/recommendations` можно было получать рекомендации.

Полный код функции recommendations приведён ниже:

In [ ]:
@app.post("/recommendations")
async def recommendations(user_id: int, k: int = 100):
    """
    Возвращает список рекомендаций длиной k для пользователя user_id
    """

    recs = rec_store.get(user_id, k)

    return {"recs": recs}

После интеграции вызовите `/recommendations` для user_id со значением `1353637`, используя код ниже (при необходимости исправьте значение recommendations_url).

Для user_id со значением `1353637` (если статус возврата равен 200) микросервис должен сгенерировать следующий словарь со списком рекомендованных item_id: `{'recs': [28187230, 27161156, 5]}`

In [ ]:
# Скрипт для обращения к сервису recommendation_service

import requests

recommendations_url = "http://127.0.0.1:8000"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 1353637, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations", headers=headers, params=params)
if resp.status_code == 200:
    recs = resp.json()
else:
    recs = []
    print(f"status code: {resp.status_code}")
    
print(recs)

__Дополнительные функции__

В сервисе можно реализовать дополнительные методы, удобные при промышленной эксплуатации, например:
- `/health`, который возвращает статус здоровья, например строку "healthy", если вызов к сервису проходит нормально, или строку "unhealthy" в противном случае.
- `/stats` — метод получения статистики от сервиса. Сервис может возвращать любую полезную статистику, которую он ведёт внутри себя (количество обработанных запросов, среднее время обработки запросов, количество ошибок и т. п.).

### Тема 5, Урок 3: Онлайн-рекомендации

__Алгоритм онлайн-рекомендаций__

Возьмём простой алгоритм онлайн-рекомендаций, который будет использовать свойство похожести объектов (item2item similarity),
когда для каждого объекта есть список объектов, которые на него похожи (в контексте поведения пользователей).
Feature Store отвечает за хранение в явном виде статичных признаков объекта — набора похожих на него айтемов.

__Шаг 1. Набор похожих объектов__

Чтобы получить набор похожих объектов, можно воспользоваться уже известным алгоритмом ALS из библиотеки implicit, у которого на такой случай есть удобный метод similar_items(), подробнее см. в [документации](https://benfred.github.io/implicit/api/models/cpu/als.html#implicit.cpu.als.AlternatingLeastSquares.similar_items).

Воспользуемся им и получим по 10 самых похожих айтемов.

__Задание 1 из 6__

Дополните код ниже, чтобы получить набор похожих объектов в similar_items. Вы можете подглядеть решение в уроке «Коллаборативная фильтрация: ALS» — там вы реализовывали похожую логику для получения персональных рекомендаций.

In [14]:
with open('models/als_model.pkl', 'rb') as fd:
    als_model = joblib.load(fd)

c:\Users\Kirill_Nosov\_Repos\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# получим энкодированные идентификаторы всех объектов, известных нам из events_train
train_item_ids_enc = events_train['item_id_enc'].unique()

max_similar_items = 10

# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items+1)

# преобразуем полученные списки в табличный формат
sim_item_item_ids_enc = similar_items[0]
sim_item_scores = similar_items[1]

similar_items = pd.DataFrame({
    "item_id_enc": train_item_ids_enc,
    "sim_item_id_enc": sim_item_item_ids_enc.tolist(), 
    "score": sim_item_scores.tolist()}) # ваш код здесь
similar_items = similar_items.explode(['sim_item_id_enc', 'score'], ignore_index=True) # ваш код здесь

# приводим типы данных
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype('int') # ваш код здесь
similar_items["score"] = similar_items["score"].astype("float")

# получаем изначальные идентификаторы
similar_items["item_id_1"] = item_encoder.inverse_transform(similar_items["item_id_enc"]) # ваш код здесь
similar_items["item_id_2"] = item_encoder.inverse_transform(similar_items["sim_item_id_enc"]) # ваш код здесь
similar_items = similar_items.drop(columns=["item_id_enc", "sim_item_id_enc"])

# убираем пары с одинаковыми объектами
similar_items = similar_items.query("item_id_1 != item_id_2")

Проверим идентификатор объекта, наиболее похожего на объект `7126`, должно получиться `7190`

In [ ]:
similar_items.query("item_id_1 == 7126").sort_values(by='score', ascending=False)

Сохраним полученный набор похожих объектов `similar_items` в файл

In [ ]:
similar_items.to_parquet("similar_items.parquet")

Полезно убедиться, что полученный набор действительно содержит похожие данные. Например, можно оценить глазами списки похожих объектов для каких-то уже известных. Создадим для этой цели функцию `print_sim_items`

In [ ]:
def print_sim_items(item_id, similar_items):
    item_columns_to_use = ["item_id", "author", "title", "genre_and_votes", "average_rating", "ratings_count"]
    
    item_id_1 = items.query("item_id == @item_id")[item_columns_to_use]
    display(item_id_1)
    
    si = similar_items.query("item_id_1 == @item_id")
    si = si.merge(items[item_columns_to_use].set_index("item_id"), left_on="item_id_2", right_index=True)
    display(si)

Можно, например, оценить похожие айтемы для следующих известных книг (числа — идентификаторы item_id):
- 7144: Ф. М. Достоевский «Преступление и наказание»;
- 16299: Агата Кристи «Десять негритят»;
- 3: Джоан Роулинг «Гарри Поттер и философский камень»;
- 18135: Уильям Шекспир «Ромео и Джульетта»;
- 17245: Брэм Стокер «Дракула».

In [ ]:
print_sim_items(7144, similar_items)

__Шаг 2. Сервис Feature Store__

Сделаем так, чтобы набор стал доступен сервису рекомендаций. Для этого создадим новый сервис, который при запуске будет загружать набор похожих объектов из файла `similar_items.parquet` и отдавать список похожих объектов через метод `/similar_items`.

__Задание 2 из 6__

Дополните код ниже, чтобы получить работоспособный сервис, возвращающий список похожих объектов через метод `/similar_items`.
Сохраните код сервиса в файле `features_service.py`

In [ ]:
# Файл features_service.py
# Запуск сервиса: uvicorn features_service:app --port 8010

import logging
from contextlib import asynccontextmanager
import pandas as pd
from fastapi import FastAPI

logger = logging.getLogger("uvicorn.error")

class SimilarItems:

    def __init__(self):

        self._similar_items = None

    def load(self, path, **kwargs):
        """
        Загружаем данные из файла
        """

        logger.info(f"Loading data, type: {type}")
        self._similar_items = pd.read_parquet(path, **kwargs) # ваш код здесь
        self._similar_items = self._similar_items.set_index('item_id_1') # ваш код здесь
        logger.info(f"Loaded")

    def get(self, item_id: int, k: int = 10):
        """
        Возвращает список похожих объектов
        """
        
        try:
            i2i = self._similar_items.loc[item_id].head(k)
            i2i = i2i[["item_id_2", "score"]].to_dict(orient="list")
        except KeyError:
            logger.error("No recommendations found")
            i2i = {"item_id_2": [], "score": []}

        return i2i


sim_items_store = SimilarItems()


@asynccontextmanager
async def lifespan(app: FastAPI):
    # код ниже (до yield) выполнится только один раз при запуске сервиса
    sim_items_store.load(
        'similar_items.parquet', # ваш код здесь
        columns=["item_id_1", "item_id_2", "score"],
    )
    logger.info("Ready!")
    # код ниже выполнится только один раз при остановке сервиса
    yield


# создаём приложение FastAPI
app = FastAPI(title="features", lifespan=lifespan)


@app.post("/similar_items")
async def recommendations(item_id: int, k: int = 10):
    """
    Возвращает список похожих объектов длиной k для item_id
    """

    i2i = sim_items_store.get(item_id, k)

    return i2i

Запустим данный сервис на порте 8010, чтобы избежать конфликта с сервисом рекомендаций:

`uvicorn features_service:app --port 8010`

Ниже пример кода для тестирования. После его запуска должно получиться:

`{'item_id_2': [480204, 51496, 2623], 'score': [0.9245510697364807, 0.9044632911682129, 0.9016129374504089]}`

In [ ]:
# Скрипт для обращения к сервису features_service

import requests

features_store_url = "http://127.0.0.1:8010"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"item_id": 17245}

resp = requests.post(features_store_url +"/similar_items", headers=headers, params=params)
if resp.status_code == 200:
    similar_items = resp.json()
else:
    similar_items = None
    print(f"status code: {resp.status_code}")
    
print(similar_items)

__Шаг 3. Сервис Event Store__

Чтобы выполнить второй пункт алгоритма («для онлайн-взаимодействия пользователя с каким-то объектом можно использовать список похожих на него объектов»), необходим компонент, умеющий сохранять и выдавать последние события пользователя, — это Event Store. 

Реализуем его также в виде сервиса. В данном случае под взаимодействием пользователя с объектом будем подразумевать любое положительное событие, например: просмотр страницы с книгой, лайк, добавление в избранное и т. п.

__Задание 3 из 6__

Дополните код сервиса так, чтобы он по методу `/put` сохранял пару значений user_id и item_id как событие, а по методу `/get` возвращал события (первыми — самые последние).

Сохраните код сервиса в файле `events_service.py`

In [ ]:
# Файл events_service.py
# Запуск сервиса: uvicorn events_service:app --port 8020

from fastapi import FastAPI

class EventStore:

    def __init__(self, max_events_per_user=10):

        self.events = {}
        self.max_events_per_user = max_events_per_user

    def put(self, user_id, item_id):
        """
        Сохраняет событие
        """

        user_events = self.events.get(user_id) # ваш код здесь
        self.events[user_id] = [item_id] + user_events[: self.max_events_per_user]

    def get(self, user_id, k):
        """
        Возвращает события для пользователя
        """
        user_events = self.events.get(user_id)[:k] # ваш код здесь

        return user_events


events_store = EventStore() # ваш код здесь

# создаём приложение FastAPI
app = FastAPI(title="events")


@app.post("/put")
async def put(user_id: int, item_id: int):
    """
    Сохраняет событие для user_id, item_id
    """

    events_store.put(user_id, item_id)

    return {"result": "ok"}


@app.post("/get")
async def get(user_id: int, k: int = 10):
    """
    Возвращает список последних k событий для пользователя user_id
    """

    events = events_store.get(user_id, k)

    return {"events": events}

Запустим данный сервис на порте 8020, чтобы он не конфликтовал с другими сервисами:

`uvicorn events_service:app --port 8020`

Ниже скрипт для сохранения взаимодействия пользователя user_id с объектом item_id:

In [ ]:
# Обращение к сервису events_service (/put)

import requests

events_store_url = "http://127.0.0.1:8020"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 1337055, "item_id": 17245}

resp = requests.post(events_store_url + "/put", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")
    
print(result)

Ниже скрипт для получения списка последних событий пользователя user_id:

In [ ]:
# Обращение к сервису events_service (/get)

import requests

events_store_url = "http://127.0.0.1:8020"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 1337055}

resp = requests.post(events_store_url + "/get", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")
    
print(result)

Проверим, что для пользователя `1127794` в Event Store нет никаких событий

In [ ]:
params = {"user_id": 1127794}
resp = requests.post(events_store_url + "/get", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")

print(result)

Затем сохраним для этого пользователя последовательно четыре события с объектами: `18734992`, `18734992`, `7785`, `4731479`

In [ ]:
params = {"user_id": 1127794, "item_id": 18734992}
requests.post(events_store_url + "/put", headers=headers, params=params)

params = {"user_id": 1127794, "item_id": 18734992}
requests.post(events_store_url + "/put", headers=headers, params=params)

params = {"user_id": 1127794, "item_id": 7785}
requests.post(events_store_url + "/put", headers=headers, params=params)

params = {"user_id": 1127794, "item_id": 4731479}
requests.post(events_store_url + "/put", headers=headers, params=params)

После чего запросим для того же пользователя последние три события. В результате должно получиться: 

`{'events': [4731479, 7785, 18734992]}`

In [ ]:
resp = requests.post(events_store_url + "/get", 
                     headers=headers, 
                     params={"user_id": 1127794, "k": 3})
print(resp.json())

__Шаг 4. Доработка сервиса рекомендаций__

Доработаем уже имеющийся код сервиса из предыдущего урока так, чтобы он позволял получить онлайн-рекомендации для последнего события пользователя. Для этого добавьте в код сервиса рекомендаций новые переменные, указывающие на адреса новых сервисов:

In [1]:
features_store_url = "http://127.0.0.1:8010"
events_store_url = "http://127.0.0.1:8020"

Затем реализуйте новый метод `/recommendations_online`, который будет выдавать список похожих объектов для последнего события пользователя (если оно есть).

__Задание 4 из 6__

Дополните код нового метода так, чтобы для последнего события пользователя, если оно есть, возвращался список похожих объектов. Это и будут онлайн-рекомендации.

In [ ]:
# Функция recommendations_online сервиса recommendation_service (v1)

@app.post("/recommendations_online")
async def recommendations_online(user_id: int, k: int = 100):
    """
    Возвращает список онлайн-рекомендаций длиной k для пользователя user_id
    """

    headers = {"Content-type": "application/json", "Accept": "text/plain"}

    # получаем последнее событие пользователя
    params = {"user_id": user_id, "k": 1}
    resp = requests.post(events_store_url + "/get", headers=headers, params=params)
    events = resp.json()
    events = events["events"]

    # получаем список похожих объектов
    if len(events) > 0:
        item_id = events[0]
        params = {"item_id": item_id, "k": k}
        # ваш код здесь (начало)
        resp = requests.post(features_store_url +"/similar_items", headers=headers, params=params)
        item_similar_items = resp.json()
        item_similar_items = item_similar_items['item_id_2']
        # ваш код здесь (конец)
        recs = item_similar_items[:k]
    else:
        recs = []

    return {"recs": recs}

Протестируйте работу нового метода. Получите онлайн-рекомендации (длиной 3) для пользователя `1291248`:

In [ ]:
params = {"user_id": 1291248, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs) 

Список пустой. Это ожидаемо, так как для пользователя в Event Store пока нет никаких событий, чтобы по ним получить онлайн-рекомендации.

Добавим событие:

In [ ]:
params = {"user_id": 1291248, "item_id": 17245}

resp = requests.post(events_store_url + "/put", headers=headers, params=params)

И снова получим онлайн-рекомендации для пользователя `1291248`. Должно получиться: `{'recs': [480204, 51496, 2623]}`

In [ ]:
params = {"user_id": 1291248, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs)

__Шаг 5. Добавим разнообразия в онлайн-рекомендации__

Текущая реализация онлайн-рекомендаций позволяет получать их только для последнего события, что очень просто и прямолинейно: если пользователь просмотрит новую книгу, то у него сменятся все онлайн-рекомендации.

Доработаем алгоритм так, чтобы онлайн-рекомендации учитывали три последних объекта, с которыми взаимодействовал пользователь. Например, так:
- Для каждого события из последних трёх получим список похожих объектов.
- Объединим полученные списки по какому-то правилу. Правило выберем простое: все полученные похожие объекты сортируются по убыванию `score`, а из упорядоченного списка удаляются дубликаты, оставляя только первое вхождение.

__Задание 5 из 6__

Дополните новую версию реализации метода `/recommendations_online` так, чтобы онлайн-рекомендации возвращались для трёх последних событий.

In [ ]:
# Функция recommendations_online сервиса recommendation_service (v2)

def dedup_ids(ids):
    """
    Дедублицирует список идентификаторов, оставляя только первое вхождение
    """
    seen = set()
    ids = [id for id in ids if not (id in seen or seen.add(id))]

    return ids


@app.post("/recommendations_online")
async def recommendations_online(user_id: int, k: int = 100):
    """
    Возвращает список онлайн-рекомендаций длиной k для пользователя user_id
    """

    headers = {"Content-type": "application/json", "Accept": "text/plain"}

    # получаем список последних событий пользователя, возьмём три последних
    params = {"user_id": user_id, "k": 3}
    # ваш код здесь (начало)
    resp = requests.post(events_store_url + "/get", headers=headers, params=params)
    events = resp.json()
    events = events['events']
    # ваш код здесь (конец)

    # получаем список айтемов, похожих на последние три, с которыми взаимодействовал пользователь
    items = []
    scores = []
    for item_id in events:
        # для каждого item_id получаем список похожих в item_similar_items
        # ваш код здесь (начало)
        params = {"item_id": item_id, "k": k}
        resp = requests.post(features_store_url +"/similar_items", headers=headers, params=params)
        item_similar_items = resp.json()
        # ваш код здесь (конец)
        items += item_similar_items["item_id_2"]
        scores += item_similar_items["score"]
    
    # сортируем похожие объекты по scores в убывающем порядке
    # для старта это приемлемый подход
    combined = list(zip(items, scores))
    combined = sorted(combined, key=lambda x: x[1], reverse=True)
    combined = [item for item, _ in combined]

    # удаляем дубликаты, чтобы не выдавать одинаковые рекомендации
    recs = dedup_ids(combined)

    return {"recs": recs}

Протестируем метод для пользователя `1291248`, сгенерировав для него несколько событий и получив пять онлайн-рекомендаций.

In [ ]:
headers = {"Content-type": "application/json", "Accept": "text/plain"}

user_id = 1291248
event_item_ids = [41899, 102868, 5472, 5907]

for event_item_id in event_item_ids:
    resp = requests.post(events_store_url + "/put", 
                         headers=headers, 
                         params={"user_id": user_id, "item_id": event_item_id})
                         
params = {"user_id": user_id, 'k': 5}

resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs)

Должно получиться: `{'recs': [608474, 3590, 8921, 194373, 736131]}`

Учитывая утверждение, что офлайн- и онлайн-рекомендации не исключают, а дополняют друг друга, доработаем сервис рекомендаций так, чтобы результат его работы мог включать в себя оба типа.

Предлагаем такую простую схему:
- онлайн-рекомендации занимают нечётные места,
- офлайн-рекомендации занимают чётные места.

Подобные схемы расстановок разнотипных элементов по различным местам ещё называют «смешиванием» (англ. blending).

Начнём с того, что переименуем метод `/recommendations` и его функцию в `recommendations_offline/`. Код функции тот же.

И реализуем новый метод `/recommendations` — уже как объединяющий оба типа рекомендаций.

__Задание 6 из 6__

Доработайте код обновлённого метода `/recommendations` так, чтобы реализовать предложенную выше схему блендинга.

In [ ]:
@app.post("/recommendations")
async def recommendations(user_id: int, k: int = 100):
    """
    Возвращает список рекомендаций длиной k для пользователя user_id
    """

    recs_offline = await recommendations_offline(user_id, k)
    recs_online = await recommendations_online(user_id, k)

    recs_offline = recs_offline["recs"]
    recs_online = recs_online["recs"]

    recs_blended = []

    min_length = min(len(recs_offline), len(recs_online))
    offline_idx = online_idx = 0
    # чередуем элементы из списков, пока позволяет минимальная длина
    for i in range(2 * min_length):
        # ваш код здесь
        if i % 2 == 0:
            recs_blended.append(recs_offline[offline_idx])
            offline_idx += 1
        else:
            recs_blended.append(recs_online[online_idx])
            online_idx += 1

    # добавляем оставшиеся элементы в конец
    # ваш код здесь
    if len(recs_offline) >= len(recs_online):
        recs_blended.extend(recs_offline[offline_idx:])
    else:
        recs_blended.extend(recs_online[online_idx:])

    # удаляем дубликаты
    recs_blended = dedup_ids(recs_blended)
    
    # оставляем только первые k рекомендаций
    # ваш код здесь
    recs_blended = recs_blended[:k]

    return {"recs": recs_blended}

На примере пользователя `1291250` протестируем доработанный сервис.

Сначала сгенерируем онлайн-события:

In [ ]:
user_id = 1291250
event_item_ids =  [7144, 16299, 5907, 18135]

headers = {"Content-type": "application/json", "Accept": "text/plain"}

for event_item_id in event_item_ids:
    resp = requests.post(events_store_url + "/put", 
                         headers=headers, 
                         params={"user_id": user_id, "item_id": event_item_id})

Получим 10 рекомендаций каждого типа для данного пользователя:

In [ ]:
headers = {"Content-type": "application/json", "Accept": "text/plain"}
params = {"user_id": 1291250, 'k': 10}

resp_offline = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)
resp_online = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
resp_blended = requests.post(recommendations_url + "/recommendations", headers=headers, params=params)

recs_offline = resp_offline.json()["recs"]
recs_online = resp_online.json()["recs"]
recs_blended = resp_blended.json()["recs"]

print(recs_offline)
print(recs_online)
print(recs_blended)

Качество рекомендаций также можно оценить выборочно: посмотрев, что рекомендации книг в целом адекватны, по авторам, названиям. Пример кода для этого приведён ниже.

In [ ]:
def display_items(item_ids):

    item_columns_to_use = ["item_id", "author", "title", "genre_and_votes", "average_rating", "ratings_count"]
    
    items_selected = items.query("item_id in @item_ids")[item_columns_to_use]
    items_selected = items_selected.set_index("item_id").reindex(item_ids)
    items_selected = items_selected.reset_index()
    
    display(items_selected)
    
    
print("Онлайн-события")
display_items(event_item_ids)
print("Офлайн-рекомендации")
display_items(recs_offline)
print("Онлайн-рекомендации")
display_items(recs_online)
print("Рекомендации")
display_items(recs_blended)

In [ ]:
#WIP